Data Verification

In [ ]:
table_name = "pi3b_asbuilt_nopfc17500ab_2022-06-28"

1. Check for holes in the data

In [6]:
import os
import sys

script_dir = os.path.abspath('')
mymodule_dir = os.path.join( script_dir, '..', 'flagships', 'post_processing' )
sys.path.append(mymodule_dir)

import GenerateColumns

ModuleNotFoundError: No module named 'mysql'

2. Check for data discontinuities

In [ ]:
sys